In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import datetime
import csv
import time

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Preparation

The concept in this part is to find the anomalies and then use the chrono of events to find the dates of anomalies. first we import the anomaly file

In [3]:
col_df = pd.read_csv("chrono_anomalies_de_fabrication_vehicule.csv", sep=";")
df = pd.read_csv("defauts_PY_2016.csv", header=None, sep=';', names=col_df.columns)
print df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164346 entries, 0 to 1164345
Data columns (total 33 columns):
USINE               1164346 non-null object
ID_VEH              1164346 non-null int64
FAMILLE_VEH         1164346 non-null object
ID_CLE_QUA          1164346 non-null object
CODE_CIRC_ANO       1164346 non-null object
LIB_CIRC_ANO        1164346 non-null object
CODE_LOC_ANO        1164346 non-null object
LIB_LOC_ANO         1164346 non-null object
CODE_NAT_ANO        1164346 non-null int64
LIB_NAT_ANO         1164346 non-null object
CODE_FAM_NAT_ANO    1164346 non-null object
LIB_FAM_NAT_ANO     1164346 non-null object
DATE_DET_ANO        1164346 non-null object
CODE_OPE_ANO        1164346 non-null int64
ZONE_OPE_ANO        851221 non-null object
TOP_UEP             1164346 non-null object
NIV_DETECT          1164346 non-null object
ARBO_STRUCT_MGMT    1164346 non-null object
CODE_LOC_ANA        1164346 non-null object
LIB_LOC_ANA         1164346 non-null object
COD_NAT_ANA

This is where some of the intelligence is. The RSP column contains the UEP where the problem is supposed to have happen. However, in the event table, there is no reference to the UEP. There are only doors, where the vehicules pass. This is why I had to find, mainly by interrogating people from PSA the correspondance between UEP and so-called doors. 

In [4]:
USABLE_DETAILED_UEPS = ["MON/1/1HC/1HC1", "MON/1/1HC/1HC2", "MON/1/1HC/1HC3", "MON/1/1HC/1HC4", "MON/1/1MV/1MV1", "MON/1/1MV/1MV3", "MON/1/1MV/1MV2", "MON/1/1MV/1MV4"]
PORTE_EPS_TABLE = {
    "MON/1/1HC/1HC1": ["ENHC01", "BR1101"],
	"MON/1/1HC/1HC2": ["BR2201", "BR2301"],
	"MON/1/1HC/1HC3": ["BR2301"],
	"MON/1/1HC/1HC4": ["BR3401", "BR3501"],
	"MON/1/1MV/1MV1": ["EPOM01", "POM101"],
	"MON/1/1MV/1MV2": ["MV1101", "MV2201"],
	"MON/1/1MV/1MV3": ["MV2201", "MV2301"],
	"MON/1/1MV/1MV4": ["MV2401", "MV3501"],
	"QCP": ["SOMV01", "SACC02"],
	"PEI": ["DETA01", "SCLA02"],
	"FER": ["SFER01", "EFER02"]
}
ALL_DOORS = []
for doors in PORTE_EPS_TABLE.itervalues():
    ALL_DOORS += doors


All databases had problem with dates. Therefore there was a need for this complicated function

In [5]:
def str_to_date(date):
    try:
        return datetime.datetime.strptime(date[:19], '%Y-%m-%dT%H:%M:%S')
    except:
        try:
            return datetime.datetime.strptime(date[:19], '%d/%m/%Y %H:%M')
        except:
            
            return datetime.datetime.strptime(date[:19], '%d/%m/%Y %H:%M:%S')

df["date_edo"] = df["DATE_DET_ANO"].map(str_to_date)

A quick look at anomalies by month

In [6]:
def to_month(date):
    return date.month
df["month"] = df["date_edo"].map(to_month)

In [7]:
df["month"].value_counts()

6     135502
11    111638
3     107443
4     105980
2     102164
9      98847
5      98219
1      96477
12     94876
10     77707
7      75235
8      60258
Name: month, dtype: int64

In the anomaly file, UEPS where very detailed. Some of them are not relevant at all, with very few anomalies. For some of them like the one of FERRAGE have no accuracy in term of doors. So I had to group them by larger UEPs, taking only the top ref. Ferrage, Peinture, and Quality are the one aggregated. For the montage, I could keep the detail.

In [8]:
def transform_UEPs(UEP):
    splitted = UEP.split("/")
    if splitted[0] in ["FER", "PEI", "QCP"]:
        return splitted[0]
    elif UEP in USABLE_DETAILED_UEPS:
        return UEP
    else: 
        return "NOT USED"
df["pb_uep"] = df["RSP"].map(transform_UEPs)

full_df = df[df["pb_uep"] != "NOT USED"]
print len(df)
print len(full_df)


1164346
966496


Then I build a subtable from the dataset of vehicule chronology, by taking only the occurence with a door which is concerned by our study. 

In [9]:
col_df = pd.read_csv("chrono_flux_vehicule_dans_usine.csv", sep=";")
columns = col_df.columns[:-1]
del col_df
full_res = {}
nb_done = 0
with open("passages_PY.csv", mode='r') as infile:

    filereader = csv.reader(infile, delimiter=";", )
    for line in filereader:
        nb_done += 1
        if nb_done % 1000000 == 0:
            print "done %s" % nb_done
        if line[3] in ALL_DOORS:
            res_dict = {}

            for i in range(len(columns)):
                res_dict[columns[i]] = line[i]
            line_id = "%s%s" % (res_dict["id_veh"], res_dict["porte"])
            full_res[line_id] = res_dict["date_passage"]


done 1000000
done 2000000
done 3000000
done 4000000
done 5000000
done 6000000
done 7000000
done 8000000
done 9000000


In [10]:
full_df["index"] = full_df.index

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


This is where the join is done. For each anomaly, we look at the table we just built in order to find the real date of apparition of the anomaly. 
when we have a door in and a door out, we are able to take the mean of the two dates.

In [11]:
def find_error_date(df_line):
    doors = PORTE_EPS_TABLE[df_line["pb_uep"]]
    dates = []

    if df_line["index"] % 100000 == 0:

        print "done %s" % str(df_line["index"])
    for door in doors:
        res_id = "%s%s" % (df_line["ID_VEH"], door)
        if not full_res.get(res_id):
            continue
        dates.append(str_to_date(full_res[res_id]))
    if len(dates) == 1:
        return dates[0]
    elif len(dates) == 2:

        return dates[0] + (dates[1] - dates[0])/2
    else:
        return 'no date'
    
full_df["date_error"] = full_df.apply(find_error_date, axis=1)

done 0
done 100000
done 200000
done 400000
done 500000
done 600000
done 700000
done 800000
done 900000
done 1000000
done 1100000


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Some of the doors could not be find. It concerns a minority. We'll just ignore this.

In [12]:
cleaned_df = full_df[full_df["date_error"] != "no date"]
def transform_datetime(date):
    return date.to_datetime()
cleaned_df.loc["date_edo"] = cleaned_df["date_edo"].map(transform_datetime)


/usr/local/lib/python2.7/site-packages/pandas/core/series.py:2177: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  new_values = map_f(values, arg)
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
cleaned_df = cleaned_df[cleaned_df["pb_uep"].notnull()]

This diff date is really interesting for PSA. It is the amount of time between an error happens and when it is detected. I did some analysis of this which are not relevant This could be used for further data cleaning. 

In [14]:
def do_diff(df_line):

        return (df_line["date_edo"] - df_line["date_error"]).total_seconds()

cleaned_df["diff_date"] = cleaned_df.apply(do_diff, axis=1)

In [15]:
del full_df
del df
del full_res

In [16]:
uep_df = cleaned_df[cleaned_df["pb_uep"] == "MON/1/1HC/1HC2"]
print len(uep_df)

80295


In [17]:
def datetime_to_hours(date):
    return date - datetime.timedelta(minutes=date.minute, seconds=date.second, microseconds=date.microsecond)
uep_df["hour_date"] = uep_df["date_error"].map(datetime_to_hours)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [18]:
grouped = uep_df[["hour_date", "pb_uep", "date_error"]].groupby("hour_date", axis=0)
ts_df = pd.concat([grouped["pb_uep"].count(), grouped["date_error"].first()], axis=1)

new_index = pd.date_range(min(ts_df.index), max(ts_df.index), freq='H')
ts_df = ts_df.reindex(new_index, fill_value=0)
zero_date = ts_df["date_error"].value_counts().index[0]

ts_df["date_error"][ts_df["date_error"] == zero_date] = ts_df[ts_df["date_error"] == zero_date].index


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# LEARNING

In [19]:
from scipy import stats
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt
print ts_df.info()
from sklearn.svm import SVR
# remove week-ends

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4935 entries, 2016-06-01 00:00:00 to 2016-12-23 14:00:00
Freq: H
Data columns (total 2 columns):
pb_uep        4935 non-null int64
date_error    4935 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 115.7 KB
None


In [20]:
def ret_weekday(date):
    return date.weekday()
ts_df["weekday"] = ts_df["date_error"].map(ret_weekday)
def ret_hour_of_day(date):
    return date.hour
ts_df["day_hour"] = ts_df["date_error"].map(ret_hour_of_day)
print len(ts_df)
small_ts_df = ts_df.drop(pd.date_range(datetime.datetime(2016, 7, 14), datetime.datetime(2016, 8, 15), freq='H'))
print len(small_ts_df)
small_ts_df = small_ts_df[small_ts_df["weekday"].isin([0, 1, 2, 3, 4])]


print np.mean(small_ts_df["pb_uep"])

4935
4166
23.4406104844


In [21]:
for hour_of_day in small_ts_df["day_hour"].unique():
    print "for hour %s" % hour_of_day
    print np.mean(small_ts_df[small_ts_df["day_hour"] == hour_of_day]["pb_uep"])

for hour 0
25.896
for hour 1
30.6984126984
for hour 2
31.1031746032
for hour 3
24.0238095238
for hour 4
30.4523809524
for hour 5
28.6746031746
for hour 6
27.8333333333
for hour 7
23.3412698413
for hour 8
23.6984126984
for hour 9
25.6746031746
for hour 10
23.873015873
for hour 11
23.4841269841
for hour 12
22.6507936508
for hour 13
22.4523809524
for hour 14
22.873015873
for hour 15
20.544
for hour 16
23.456
for hour 17
23.008
for hour 18
19.248
for hour 19
21.112
for hour 20
6.536
for hour 21
8.344
for hour 22
22.656
for hour 23
30.68


For the learning I had to limit myself for shifts. Indeed for errors to be reported, we need to wait some time (3), so I couldn't use shorter shift to predict. I brought a shift of 8, because it corresponds to the amount of hours a worker stays on an unit. A 1 day shift is pretty classic. Finally, I added a parameter in order to be able to say if it is night or not, because I have observed that during night, the errors are higher. 

In [22]:
small_ts_df["shift3h"] = small_ts_df["pb_uep"].shift(3)
small_ts_df["night"] = small_ts_df["day_hour"].isin(range(5) + [23])
small_ts_df["shift1d"] = small_ts_df["pb_uep"].shift(24)
small_ts_df["shift8h"] = small_ts_df["pb_uep"].shift(8)
learning_df = small_ts_df[small_ts_df["shift1d"].notnull()]
print learning_df.info()
X = learning_df[["shift1d", "shift8h", "shift3h", "night"]].values
Y = learning_df["pb_uep"].values

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2990 entries, 2016-06-02 00:00:00 to 2016-12-23 14:00:00
Data columns (total 8 columns):
pb_uep        2990 non-null int64
date_error    2990 non-null datetime64[ns]
weekday       2990 non-null int64
day_hour      2990 non-null int64
shift3h       2990 non-null float64
night         2990 non-null bool
shift1d       2990 non-null float64
shift7d       2990 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(3), int64(3)
memory usage: 189.8 KB
None


Model testing. With cross-validation specific to time series. Iterated a lot over the model or variables. Regression and bayesian regression seems to work the best. There are still some models that we could still test. 

In [23]:
def test_model(X, Y, model):
    tscv = TimeSeriesSplit(n_splits=5)
    err = []
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train, y_train)
        err.append(sqrt(mean_squared_error(y_test, model.predict(X_test))))

        plt.show()
    print np.mean(err)
test_model(X, Y, LinearRegression(normalize=True))
test_model(X, Y, BayesianRidge(normalize=True))

14.7282336865
14.7359835851


/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [24]:
test_model(X, Y, SVR(kernel='rbf', C=1e3, gamma=0.1))
test_model(X, Y, SVR())


17.5840582996
15.491265582


In [117]:
# mock data 
import numpy
mean = 0
std = 2
num_samples = 24
samples = numpy.random.normal(mean, std, size=num_samples)
predicted = ts_df["count"][0:24].values + samples
print {
    "past_values": list(ts_df["count"][0:16].values),
    "predicted": list(predicted),
    "past_shift": 238,
    "next_shift": 150
}
    

{'past_shift': 238, 'next_shift': 150, 'predicted': [43.390493300666535, 102.43893438658446, 66.146168058345353, 21.646164531258275, 32.151470477770218, 24.442319958366035, 47.051170766997878, 33.276233762629815, 28.13650158588111, 32.641421141127054, 24.615585681238546, 18.005537337068098, 24.409293884248875, 19.307337790491815, 20.840342168091791, 16.163386880064738, 28.061571005696411, 25.588152341241177, 11.371529128070271, 17.085130959018738, 25.403214626484402, 17.562240759670562, 32.503064048617148, 60.069352130768543], 'past_values': [42, 99, 68, 21, 33, 20, 43, 35, 28, 35, 23, 19, 25, 22, 21, 14]}


In [234]:
import numpy
mean = 0
std = 2
num_samples = 24
samples = numpy.random.normal(mean, std, size=num_samples)

df.drop()

NameError: name 'df' is not defined

In [64]:
user = "pugetp@gmail.com"
password= "Tifous1989"

In [109]:
from requests.auth import HTTPBasicAuth
url="https://psa.opendatasoft.com/api/records/1.0/search/?dataset=chrono_flux_vehicule_dans_usine&rows=1000&q=column_2%3D5126792&sort=column_7&facet=journee_prod&facet=porte&facet=column_7&facet=column_2&facet=column_5"
rep = requests.get(url, auth=HTTPBasicAuth(user, password))
print rep


<Response [200]>


In [58]:
"http://psa.opendatasoft.com/api/records/1.0/search/?apikey=%s?dataset=chrono_flux_vehicule_dans_usine&facet=journee_prod&facet=porte&facet=column_7" % api_key

'http://psa.opendatasoft.com/api/records/1.0/search/?apikey=3b74cc5d9284b4f2e2dae0dddcc2d9e00c6333b5d941f1037aacf326?dataset=chrono_flux_vehicule_dans_usine&facet=journee_prod&facet=porte&facet=column_7'

In [176]:
df["RSP"].value_counts()

MON/1/1HC/1HC2        82312
MON/1/1MV/1MV4        51368
MON/1/1HC/1HC4        49958
MON/1/1HC/1HC3        46412
MON/1/1MV/1GAV        45604
MON/1/1HC/1HC1        40751
MON/1/1MV/1MV2        38590
MON/1/1MV/1MV1        37010
FER                   32749
MON/1/1MV/1MV3        30665
MON/1/1MV/1MEC        30180
QCP/QUA/CRAC/LAC      28047
FER/GRXX/GR40/MLF1    18913
FER/GRXX/GR40/MEF1    15942
PEI/-/ASP/LAQ         13624
FER/GRXX/GR40/MEF2    10161
MON/1/1HC/1POR         9162
FER/GRXX/GR30/ARM      7593
QCP/QUA/CRAC/BAEB      6748
MON/1/1HC/1KIT         5885
QCP/QUA/CRAC/CVM       5712
CPL                    5134
EMB                    4277
QCP/QUA/AVCF/AVCF      4254
QCP/QUA/CRAC/LOCA      3951
MON/-/MAI              2721
FER/GRXX/GR20          2606
QFE                    2516
QCP/QUA/BTU/BU         2509
PEI/-/FOND/CATA        2122
                      ...  
PEI/-/ASP               324
FER/GRXX/GR46/OUA9      294
PEI/-/ASP/LM            284
PEI/-/ASP/BIMA          258
PEI/-/ASP/BIFI      

In [136]:

nb_done = 0
id_vec = []
for index, line in df.iterrows():
    nb_done += 1
    if nb_done % 10 == 0:
        break
    id_vec.append(line["ID_VEH"])

url="https://psa.opendatasoft.com/api/records/1.0/search/?dataset=chrono_flux_vehicule_dans_usine&rows=1000%s&sort=column_7&facet=journee_prod&facet=porte&facet=column_7&facet=column_2&facet=column_5"

url += "&q=column_2%%3D%s" % ("+or+".join([str(ident) for ident in id_vec]))
print url

rep = requests.get(url, auth=HTTPBasicAuth(user, password))
print rep.content

https://psa.opendatasoft.com/api/records/1.0/search/?dataset=chrono_flux_vehicule_dans_usine&rows=1000%s&sort=column_7&facet=journee_prod&facet=porte&facet=column_7&facet=column_2&facet=column_5&q=column_2%3D5424593+or+5424379+or+5424431+or+5424449+or+5424093+or+5423902+or+5440389+or+5432570+or+5417593



<!DOCTYPE html>
<html>
<head>
    <title>OpenDataSoft &mdash; Erreur interne</title>
    <link rel="stylesheet" type="text/css" href="/static/vendor/font-awesome-4.7.0/css/font-awesome.min.css">
    <meta charset="UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <link rel="stylesheet" href="/static/compressed/css/7df55e53cb6c.css" type="text/css" />
</head>
<body class="ods-error-page">
    <div class="ods-error-page__page-container">
        <div class="ods-error-page__container">
            <img class="ods-error-page__logo" src="/static/ods/img/ods-logo-large.png">
            <h2 class="ods-error-page__title">Le serveur a rencontré une erre

In [145]:
import time
t=time.time()
url = "https://psa.opendatasoft.com/api/records/1.0/search/?dataset=chrono_flux_vehicule_dans_usine&q=column_2%3D5424593+or+5424379+or+5424431+or+5424449+or+5424093&facet=journee_prod&facet=porte&facet=column_7&facet=column_2"
rep = requests.get(url, auth=HTTPBasicAuth(user, password))
print time.time() - t
print rep

1.97086501122
<Response [200]>


In [138]:
print rep.content

{"nhits": 730, "parameters": {"dataset": ["chrono_flux_vehicule_dans_usine"], "timezone": "UTC", "q": "column_2=5424593 or 5424379 or 5424431 or 5424449 or 5424093 or 5423902 or 5440389 or 5432570 or 5417593", "rows": 10, "format": "json", "facet": ["journee_prod", "porte", "column_7", "column_2"]}, "records": [{"datasetid": "chrono_flux_vehicule_dans_usine", "recordid": "317471bd36dce12abb6998fd6974f1d1441d54e6", "fields": {"column_7": "2016-06-30T17:08:46+00:00", "journee_prod": "2016-06-30", "column_5": "Sortie ligne plateforme A5/A9", "column_2": 5417593, "column_1": "PY", "porte": "TCPA01", "column_22": "2016-07-20T18:15:41+00:00"}, "record_timestamp": "2017-01-18T15:22:53+00:00"}, {"datasetid": "chrono_flux_vehicule_dans_usine", "recordid": "83e85bd0c44f98dd1d513f92f949669049bc0f73", "fields": {"column_7": "2016-06-30T21:31:38+00:00", "journee_prod": "2016-06-30", "column_5": "Sortie ligne appariement cdc A9", "column_2": 5417593, "column_1": "PY", "porte": "APC201", "column_22":

In [ ]:
bulk_size = 10
max_iter = len(df) / bulk_size
base_url = "https://psa.opendatasoft.com/api/records/1.0/search/?dataset=chrono_flux_vehicule_dans_usine&rows=10000&q=column_2%3D"
end_url = "&facet=journee_prod&facet=porte&facet=column_7&facet=column_2"

for i in range(max_iter):
    t = time.time()
    small_df = df[i*bulk_size :(i + 1) * bulk_size]
    
    url = base_url + "+or+".join([str(id_vec) for id_vec in small_df["ID_VEH"]]) + end_url
    rep = requests.get(url, auth=HTTPBasicAuth(user, password))
    print rep
    print "request took %s" % (time.time() - t)